# Reranking with RAGatouille
Inspired by : https://github.com/AnswerDotAI/RAGatouille/blob/main/examples/04-reranking.ipynb
In this quick example, we'll use the `RAGPretrainedModel` magic class to demonstrate how to **re-rank documents** retrieved by another retriever, such as **your existing RAG pipeline**.
First, as usual, let's load up a pre-trained ColBERT model:

In [ ]:
from ragatouille import RAGPretrainedModel

# Loading a pre-trained model for reranking
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

Now that our model is loaded, we must build an index of our documents for our first retrieval step! In the real world, you'd likely already have some sort of pipeline doing this, which we're going to emulate here, using bge embeddings Spotify's excellent `voyager` library.

In [ ]:
from sentence_transformers import SentenceTransformer
from voyager import Index, Space


class MyExistingRetrievalPipeline:
    index: Index
    embedder: SentenceTransformer

    def __init__(self, embedder_name: str = "BAAI/bge-small-en-v1.5"):
        # Initialize the sentence transformer with a specific model
        self.embedder = SentenceTransformer(embedder_name)
        # Dictionary to store document contents with their index
        self.collection_map = {}
        # Create vector index using cosine similarity
        # num_dimensions matches the embedding size from the model
        self.index = Index(
            Space.Cosine,
            num_dimensions=self.embedder.get_sentence_embedding_dimension(),
        )

    def index_documents(self, documents: list[str]) -> None:
        # Process each document in the input list
        for document in documents:
            # 1. Convert document content to embedding vector
            # 2. Add embedding to index and get back an index ID
            # 3. Store original content in collection_map with index ID as key
            self.collection_map[
                self.index.add_item(self.embedder.encode(document["content"]))
            ] = document["content"]

    def query(self, query: str, k: int = 10) -> list[str]:
        # Convert query text to embedding vector
        query_embedding = self.embedder.encode(query)
        to_return = []
        # Search index for k most similar vectors
        # Get their IDs and look up original content
        for idx in self.index.query(query_embedding, k=k)[0]:
            to_return.append(self.collection_map[idx])
        return to_return

In [ ]:
existing_pipeline = MyExistingRetrievalPipeline()

Now that our mock of existing pipeline is set up, let's index some documents with it! We'll re-use our favourite combo from the previous examples: `CorpusProcessor` and `get_wikipedia_page()`:

In [ ]:
from ragatouille.data import CorpusProcessor

# Create an instance of CorpusProcessor
# This will be used to split documents into smaller, manageable chunks
corpus_processor = CorpusProcessor()

# Open and read the content of 'text_for_reranking.txt' file
with open("text_for_reranking.txt", "r") as file:
    # Read the entire content of the file into the 'content' variable
    content = file.read()

# Convert the content into a list containing a single string
# This is done because process_corpus expects a list of documents
document = [content]

# Process the document using CorpusProcessor
# chunk_size=100 means the text will be split into chunks of approximately 100 tokens
# This helps in:
# 1. Making the text more manageable for processing
# 2. Creating smaller segments that are better for retrieval
# 3. Ensuring chunks aren't too large for the model to handle effectively
document = corpus_processor.process_corpus(document, chunk_size=100)

Now, let's add those to the voyager index so we can simulate a real query:

In [ ]:
existing_pipeline.index_documents(document)

In [ ]:
query = "How does climate change affect agriculture, and what are the impacts on crop yields, farming practices, and food security?"
raw_results = existing_pipeline.query(query, k=7)
raw_results

Oh! We can see in the results that we're looking for is explained very clearly:

>   Soil quality is also impacted by these changes, as increased heat and water stress can reduce soil fertility and increase erosion. In the long term, these changes could lead to reduced crop yields, particularly in regions already facing food security challenges.\nClimate Change and Crop Yields\nRising temperatures are expected to significantly decrease crop yields, especially for staple crops like wheat, rice, and corn. Heat stress during critical stages of crop development can result in lower yields and poorer crop quality. **3th** most relevant result! In a real RAG pipeline, this'd often be well outside the context you'd give to your LLM.

This is where ColBERT re-ranking comes into play. Let's use our previously loaded `RAGPretrainedModel` to re-rank the results of our existing pipeline:

In [ ]:
RAG.rerank(query=query, documents=raw_results, k=5)

And here it is! The relevant extract is now all the way at the top of the results, ready to be passed to the rest of your pipeline!

So why not just use rerank() on the whole index if it's so good? Well, you could, but it's not very efficient. ColBERT is an extremely fast querier, but it needs to have an index built to do so. When you're using ColBERT to rerank documents, it's doing it index-free, which means it needs to encode all your documents and queries, and perform the comparison on the fly. This is fine for a handful of document on CPU or a few hundreds on GPU, but it's going to take exponentially longer as you add more documents!

Re-ranking the results of another retrieval method is a good compromise: it allows you to leverage ColBERT's power without having to modify the rest of your pipeline, just increase the `k` value of your retriever and let ColBERT rescore them!